In [4]:
from tensorflow.keras.datasets import mnist
from vae import VAE
%config Completer.use_jedi = False

In [5]:
# global constants
LEARNING_RATE = 0.0005
BATCH_SIZE = 32
NUM_EPOCHS = 100


def load_mnist():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # apply normalization
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255
    # add channel dimension
    x_train = x_train.reshape(x_train.shape + (1,))
    x_test = x_test.reshape(x_test.shape + (1,))
    return x_train, y_train, x_test, y_test


def train(x_train, learning_rate, batch_size, num_epochs):
    autoencoder = VAE(
        input_shape=(28, 28, 1),
        conv_filters=(32, 64, 64, 64),
        conv_kernels=(3, 3, 3, 3),
        conv_strides=(1, 2, 2, 1),
        latent_space_dim=2
    )
    autoencoder.summary()
    autoencoder.compile(learning_rate)
    autoencoder.train(x_train, batch_size, num_epochs)
    return autoencoder

In [6]:
# if __name__ == "__main__":
num_samples = 10_000
x_train, _, _, _ = load_mnist()
autoencoder = train(x_train[:num_samples], LEARNING_RATE, BATCH_SIZE, NUM_EPOCHS)

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_layer_1 (Conv2D)   (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_relu_1 (ReLU)           (None, 28, 28, 32)   0           encoder_conv_layer_1[0][0]       
__________________________________________________________________________________________________
encoder_bn_1 (BatchNormalizatio (None, 28, 28, 32)   128         encoder_relu_1[0][0]             
____________________________________________________________________________________________

10000/10000 [==============================] - 80s 8ms/sample - loss: 54.0335 - _calculate_reconstruction_loss: 0.0498 - _calculate_KL_loss: 4.2550
Epoch 9/100
10000/10000 [==============================] - 74s 7ms/sample - loss: 53.4089 - _calculate_reconstruction_loss: 0.0491 - _calculate_KL_loss: 4.3481
Epoch 10/100
10000/10000 [==============================] - 63s 6ms/sample - loss: 52.9379 - _calculate_reconstruction_loss: 0.0485 - _calculate_KL_loss: 4.4125
Epoch 11/100
10000/10000 [==============================] - 60s 6ms/sample - loss: 52.5025 - _calculate_reconstruction_loss: 0.0480 - _calculate_KL_loss: 4.4559
Epoch 12/100
10000/10000 [==============================] - 63s 6ms/sample - loss: 52.3128 - _calculate_reconstruction_loss: 0.0478 - _calculate_KL_loss: 4.5341
Epoch 13/100
10000/10000 [==============================] - 62s 6ms/sample - loss: 52.0440 - _calculate_reconstruction_loss: 0.0474 - _calculate_KL_loss: 4.6540
Epoch 14/100
10000/10000 [======================

Epoch 59/100
10000/10000 [==============================] - 48s 5ms/sample - loss: 44.5252 - _calculate_reconstruction_loss: 0.0391 - _calculate_KL_loss: 5.4520
Epoch 60/100
10000/10000 [==============================] - 52s 5ms/sample - loss: 44.3610 - _calculate_reconstruction_loss: 0.0389 - _calculate_KL_loss: 5.4518
Epoch 61/100
10000/10000 [==============================] - 50s 5ms/sample - loss: 44.4416 - _calculate_reconstruction_loss: 0.0390 - _calculate_KL_loss: 5.4089
Epoch 62/100
10000/10000 [==============================] - 47s 5ms/sample - loss: 44.1575 - _calculate_reconstruction_loss: 0.0387 - _calculate_KL_loss: 5.4138
Epoch 63/100
10000/10000 [==============================] - 51s 5ms/sample - loss: 44.0540 - _calculate_reconstruction_loss: 0.0386 - _calculate_KL_loss: 5.4223
Epoch 64/100
10000/10000 [==============================] - 50s 5ms/sample - loss: 44.0110 - _calculate_reconstruction_loss: 0.0386 - _calculate_KL_loss: 5.4326
Epoch 65/100
10000/10000 [========

In [9]:
# save and re-load the VAE model
folder_path = "../trained_models/vae_model/"
autoencoder.save(folder_path)
autoencoder2 = VAE.load(folder_path)
autoencoder2.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_layer_1 (Conv2D)   (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
encoder_relu_1 (ReLU)           (None, 28, 28, 32)   0           encoder_conv_layer_1[0][0]       
__________________________________________________________________________________________________
encoder_bn_1 (BatchNormalizatio (None, 28, 28, 32)   128         encoder_relu_1[0][0]             
____________________________________________________________________________________________